## 添加依赖

In [162]:
import os
import json
import numpy as np
import faiss
from typing import List, Dict, Tuple
from dotenv import load_dotenv
from ollama import Client
from collections import deque
import requests

### 加载知识数据，包含 .index, _chunks.json, _vectors.json 三个文件
### 并且将文本与向量对应起来，返回FAISS索引和文本元数据列表。

In [163]:
def load_data_group(folder_path: str) -> Tuple[faiss.Index, List[Dict]]:
    files = os.listdir(folder_path)
    # 找出所有.index文件，依据命名前缀找到对应.json文件
    index_files = [f for f in files if f.endswith('.index')]

    # 结果：合并所有组的FAISS索引和对应元数据
    # 为了合并多个FAISS索引，我们这里采取一次性加载所有embedding并重建索引的方案（便于统一检索）
    all_embeddings = []
    all_metadata = []

    for idx_file in index_files:
        prefix = idx_file[:-6]  # 去掉 '.index'
        chunks_file = prefix + "_chunks.json"
        vectors_file = prefix + "_vectors.json"

        idx_path = os.path.join(folder_path, idx_file)
        chunks_path = os.path.join(folder_path, chunks_file)
        vectors_path = os.path.join(folder_path, vectors_file)

        # 检查对应文件是否存在
        if not (os.path.exists(chunks_path) and os.path.exists(vectors_path)):
            print(f"警告：文件组不完整，缺少 {chunks_file} 或 {vectors_file}，跳过 {prefix}")
            continue

        # 读取文本块
        with open(chunks_path, "r", encoding="utf-8") as f:
            chunks_data = json.load(f)

        # 读取向量数据
        with open(vectors_path, "r", encoding="utf-8") as f:
            vectors_data = json.load(f)

        # 将文本和向量对应起来，确保id对应一致
        id_to_text = {item['id']: item['text'] for item in chunks_data}

        for vec_item in vectors_data:
            vec_id = vec_item['id']
            embedding = vec_item.get('embedding')
            if embedding is None:
                print(f"警告：id={vec_id}在 {vectors_file} 中缺少embedding，跳过")
                continue
            text = id_to_text.get(vec_id, "")
            all_embeddings.append(embedding)
            all_metadata.append({
                "id": vec_id,
                "text": text,
                "source_file": vec_item.get("source_file", ""),
                "prefix": prefix
            })

    # 构建FAISS索引
    if len(all_embeddings) == 0:
        raise ValueError("没有加载到任何embedding向量，无法构建索引")

    dimension = len(all_embeddings[0])
    #print(f"总共加载向量数：{len(all_embeddings)}，向量维度：{dimension}")

    embeddings_np = np.array(all_embeddings, dtype=np.float32)

    # 创建FAISS索引，使用内积或欧式距离（这里默认L2距离）
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_np)
    #print(f"FAISS索引构建完成，向量总数: {index.ntotal}")

    return index, all_metadata

### embedding向量化文本

In [164]:
client = Client(host='http://localhost:11434')
 
def get_embedding(text, model="dengcao/Qwen3-Embedding-8B:Q5_K_M"):
    response = client.embeddings(model=model, prompt=text)
    return np.array(response['embedding'], dtype=np.float32)

### RAG 引擎类，检索生成问答

In [165]:
class RAGSystem:
    def __init__(self, index_path, temperature=0.8, model_name="qwen3:8b"):
        self.index, self.metadata = load_data_group(index_path)
        self.chat_history = deque(maxlen=6)  # 保留3轮对话
        self.temperature = temperature  # 存储温度值
        self.model_name = model_name  # 生成模型名称
    
    def _retrieve(self, query: str, top_k=5):
        """向量检索"""
        emb = get_embedding(query)
        distances, indices = self.index.search(np.array([emb]), top_k)
        return [self.metadata[idx] for idx in indices[0]]
    
    def _build_prompt(self, context_docs, query):
        """知识增强的Prompt构造"""
        context = "\n\n".join(f"【知识片段 {i}】{doc['text']}" 
                             for i, doc in enumerate(context_docs))
        return f"背景知识：\n{context}\n\n问题：{query}\n回答要求：用中文回答，包含知识来源"
    
    def generate_answer(self, query, top_k):
        # 1. 检索上下文
        context_docs = self._retrieve(query,top_k)


        # 2. 构造Prompt（包含多轮历史）
        prompt = "你是有专业文档支持的助手。请根据以下知识和对话历史回答问题：\n\n"
    
        # 添加对话历史（如果有）
        if self.chat_history:
            for i, content in enumerate(self.chat_history):
                role = "用户" if i % 2 == 0 else "助手"
                prompt += f"{role}: {content}\n"
    
        # 添加当前问题和知识片段
        prompt += f"\n知识背景：\n{self._build_prompt(context_docs, query)}"
        print("=== 最终 prompt ===")
        print(prompt)
        # 3. 调用Ollama
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,  # 明确要求非流式输出
                "options": {
                    "temperature": self.temperature  # 设置温度值
                }
            }
        )
        answer = response.json()["response"]
        
        # 4. 后处理
        clean_answer = self._postprocess(answer)
        self.chat_history.extend([query, clean_answer])
        return clean_answer


    def _postprocess(self, text):
        """答案优化"""
        last_query = self.chat_history[-2] if len(self.chat_history) >= 2 else ""
        # 学术文献添加引用
        if any(kw in last_query  for kw in ["文献", "研究"]):
            return f"{text}\n\n来源：{self.current_sources()}"
            
        # 法律文件精确条款标注
        elif "条款" in last_query :
            return re.sub(r"第([零一二三四五六七八九十百]+)条", r"【\g<0>】", text)
        
        return text.strip()

### 主程序

In [166]:
if __name__ == "__main__":
    folder = "/workspace/code/datas/output"  # 修改为你的文件夹路径
    rag_engine = RAGSystem(folder)

    
    q = '总结一下民国文化的特点'
    print("欢迎使用AI知识脑\n")
    ans = rag_engine.generate_answer(q, top_k=5)
    print("\n=== 回答 ===\n")
    from IPython.display import display, HTML
    display(HTML("<style>pre { white-space: pre-wrap; }</style>"))
    print(ans)

欢迎使用AI知识脑

=== 最终 prompt ===
你是有专业文档支持的助手。请根据以下知识和对话历史回答问题：


知识背景：
背景知识：
【知识片段 0】化，它在专制主义压抑下崛起，在同帝国主义和封建买办文化的搏争中壮大，直至确立在全国范围内的主导地位。 其长现之三在于：民国社会复杂的阶级构成，特别是政党斗争和政权分割的多元政治形势，决定了民国文化的多元格局，即多种文化并存、两大文化阵营对垒的局面；同时也赋了了它鲜明的政治性格。 在整个民国时期，社会结构呈现出十分复杂的状态，作为各个社会阶级和阶层意识形态表现的文化也同样呈现出比晚清时期更为错综复杂的构成。毛泽东在《新民主主义论》小，曾将此期的文化成分概括为三种：（1）“帝国主义文化”，－-切包含双化思想的文化，都属于这一类：（2）“半封建文化"，凡属上张尊孔读经、提倡旧礼教、旧思想、反对新文化新思想者，都属于这-类;（3）新民主主义的文化，即“无产阶级领导的反帝反封建的文化”。这当然是就其主要成分而言，如果分得细些，还有更多。

【知识片段 1】国最杰出的抒情诗人”。② 在诗歌创作中取得更大成就的是新月社成员，其代表人物是徐志摩、闻一多和朱湘。徐志摩（1891一1931年）是“新月诗派”的盟主，在1922年到1931年近十年的时间里，先后出版了《志摩的诗》、《翡冷翠的一夜》、《猛虎集》等诗集，《落叶》、《自剖》、《巴黎的鳞爪》等散文集。收在《志摩的诗》中的早期诗作大都内容健康，格调清新，形式活泼白然，如《落叶小唱》、《残诗》等，洋溢着积极乐观的情调。但面对当时中国社会的黑 暗，诗人也在许多诗篇中流露出失望和颓唐情绪，特别是在《翡冷翠的·夜》和《猛虎集》中的一些诗篇，如《大帅》、《三月「二深夜大沽口外》等。此后，他以全部精力追求诗的格律的改革与创造，音调的和谐与匀称。代表徐志艺术成就的，是那些并无明显社会内容的抒情诗，它们是“从性灵暖处来的诗句”。

【知识片段 2】籍也丘花八门』。这使得各教派多主张“三教归一”或“五教归一”，尽管他们对各家思想并无真正透彻的了解。需要指出的是，当时“在国内几乎做到了无上尊严的地位”的科学②，对秘密宗教也有一定的影响，以至于－-些教派如一贯道·--再强调己教“就理论言之则为哲学，玄而义玄；就具体研究，则又属科学”③。 最后，各秘密教派之所以能在民国时代此消彼长，生生不息，足

<think>
好的，我现在需要总结民国文化的特点。根据提供的知识片段，我需要仔细分析每个片段的内容，找出共同点和关键点。首先看知识片段0，里面提到民国文化在专制主义压抑下崛起，与帝国主义和封建买办文化斗争中壮大，确立了主导地位。还提到社会结构复杂，阶级构成多样，导致文化多元格局，分为多种文化并存和两大阵营对垒。毛泽东在《新民主主义论》中将文化分为帝国主义文化、半封建文化和新民主主义文化。这说明民国文化具有多元性和政治性。

接下来是知识片段1，主要讲新月社成员如徐志摩的诗歌创作，强调抒情和形式改革，还有社会内容的缺失。这可能反映出民国文化中的文学艺术追求形式美和情感表达，但可能缺乏强烈的社会批判性。不过这部分更多是文学方面的，可能需要结合其他片段看整体文化特点。

知识片段2提到秘密宗教的兴盛，各教派主张三教归一，科学对宗教的影响，以及秘密宗教的短暂生命力。这说明民国时期宗教文化有融合趋势，但存在混乱和低质量的问题，可能反映出社会动荡下宗教寻求适应和变革。

知识片段3讨论文化交流和中国化倾向，天主教和新教的中国化，如本色化、自立运动，采用中国传统节日和礼仪，甚至融合佛教元素。这显示民国文化中的宗教具有本土化、民族化的特点，强调与本土文化的结合。

知识片段4涉及新儒家哲学的发展，冯友兰和贺麟等人用西方哲学方法改造宋明理学，形成新儒家流派。这说明民国文化在思想领域有中西结合的趋势，学术界积极吸收西方哲学，同时维护传统儒家思想。

综合这些信息，民国文化的特点应该包括：多元并存（不同文化阵营）、政治性（与新民主主义相关）、宗教的本土化、中西融合（如新儒家）、社会动荡带来的文化变革等。需要确认每个知识点是否都支持这些结论，并正确引用来源。例如，知识片段0提到毛泽东的分类和多元格局，知识片段3和4涉及中西结合和宗教中国化，知识片段2提到宗教的融合和科学影响。确保每个特点都有对应的知识片段支持，并正确标注来源。
</think>

民国文化具有以下特点，依据知识来源总结如下：

1. **多元文化并存与政治性**  
   民国时期社会结构复杂，阶级和阶层意识形态呈现多维度构成，形成“多种文化并存、两大文化阵营对垒”的格局（知识片段0）。毛泽东在《新民主主义论》中将文化分为三类：帝国主义文化、半封建文化与新民主主义文化（知识片段0），凸显其政治性，即文化与阶级斗争、反帝反